**Payoff Function:**

$S_{T}^{1/3} + 1.5 \times log(S_{T}) + 10.0$

The Black-Scholes model would yield a payoff function of:

\begin{equation}
V_{0} = e^{-rT} \left[S_{0}^{1/3} e^{\frac{rT}{3} - \frac{\sigma^{2}T}{9}} + \frac{3}{2} \log\left(S_{0}\right) + \frac{3}{2}\left(r - \frac{\sigma^{2}}{2}\right)T + 10 \right]
\end{equation}

In [29]:
import numpy as np

implied_vol = 0.1807
S = 3663.3762493669747
T = 0.1232876712328767
r = 0.002206
K = 3663.3898755765863
alpha = 2.137298210993995
beta = 0.7
rho = -0.6008211054152619
nu = 2.2665005058006495

def BlackScholesPayoff(S, r, sigma, T):
    return np.exp(-r*T)*(S**(1/3)*np.exp(r*T/3 - sigma**2*T/9) + 3/2*np.log(S) + 3/2*(r - sigma**2/2)*T + 10)

BlackScholesPayoff(S,r,implied_vol,T)

37.70663131596357

The Bachelier model would yield a payoff function of:

\begin{equation}
V_{0} = e^{-rT} \left[S_{0}^{1/3} + 1.5\log(S_{0}) - \frac{3\sigma^{2}T}{4S_{0}^{2}} + 10 \right]
\end{equation}

In [32]:
def BachelierPayoff(S, r, sigma, T):
    return np.exp(-r*T)*(S**(1/3) + 1.5*np.log(S) - (3*sigma**2*T)/(4*S**2) + 10)
BachelierPayoff(S,r,implied_vol,T)

37.71477093059557

Static Replication of european payoff (SABR):

\begin{equation}
e^{-rT} h\left(F\right) + \int_{0}^F h''(K) P\left(K\right)dK + \int_{F}^\infty h''(K) C\left(K\right)dK\\
h\left(F\right) = S_{0}^{\frac{1}{3}} e^{\frac{rT}{2}} +1.5\log(S_{0}e^{rT}) + 10 \hspace{1cm} h\left(S_{T}\right) = S_{T}^{1/3} + 1.5\log\left(S_{T}\right) + 10 \hspace{1cm} h'\left(S_{T}\right) = \frac{1}{3S_{T}^{\frac{2}{3}}} + \frac{3}{2S_{T}} \hspace{1cm} h''\left(S_{T}\right) = - \frac{2}{9S_{T}^{\frac{5}{3}}} - \frac{3}{2S_{T}^{2}}
\end{equation}

In [31]:
from scipy.stats import norm
from scipy.integrate import quad

def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

def BlackScholesCall(S, K, r, sigma, T):
    d1 = (np.log(S/K)+(r+sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)

def BlackScholesPut(S, K, r, sigma, T):
    return BlackScholesCall(S, K, r, sigma, T) - S + K*np.exp(-r*T)

def SABRCall(S, K, r, T, alpha, beta, rho, nu):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)

def SABRPut(S, K, r, T, alpha, beta, rho, nu):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)

def SABRcallintegrand_sr(S, K, r, T, alpha, beta, rho, nu):
    price = (- (2/(9*(K**(5/3)))) - (3/(2*K**2))) * SABRCall(S, K, r, T, alpha, beta, rho, nu)
    return price

def SABRputintegrand_sr(S, K, r, T, alpha, beta, rho, nu):
    price = (- (2/(9*(K**(5/3)))) - (3/(2*K**2))) * SABRPut(S, K, r, T, alpha, beta, rho, nu)
    return price

def SABRStaticReplication(S, K, r, T, alpha, beta, rho, nu):
    F = S * np.exp(r*T)
    h_F = S**(1/3)*np.exp(r*T/3) + 1.5*np.log(S*np.exp(r*T)) + 10
    I_put = quad(lambda x: SABRcallintegrand_sr(x, S, r, T, alpha, beta, rho, nu), 1e-6, F)
    I_call = quad(lambda x: SABRputintegrand_sr(x, S, r, T, alpha, beta, rho, nu), F, 5000)
    return np.exp(-r*T)*h_F + I_put[0] + I_call[0]

SABRStaticReplication(S, K, r, T, alpha, beta, rho, nu)

37.70204130640218

**Payoff Function:**

\begin{equation*}
  \begin{split}
    \sigma^2{_M}{_F}T &= \mathbb{E}\left[ \int_0^T \sigma_t^2 \; dt \right]
  \end{split}
\end{equation*}

The Black-Scholes and Bachelier Models would yield a payoff function of:


\begin{equation*}
  \begin{split}
    e^{-rT} \mathbb{E}\left[ \int_0^T \sigma_t^2 \; dt \right] &= e^{-rT} \int_0^T \sigma^2_t \; dt \\
    &= e^{-rT} \sigma^2_tT
  \end{split}
\end{equation*}

In [33]:
implied_vol = 0.1807
r = 0.002206
T = 0.1232876712328767

def BlackScholesBachelierPayoff(implied_vol,r, T):
    payoff = np.exp(-r*T)*(implied_vol**2)*T
    return payoff

BlackScholesBachelierPayoff(implied_vol, r, T)

0.004024554734568931

Static Replication of European Payoff:
    \begin{equation*}
\begin{split}
    e^{-rT} \mathbb{E}\left[\int_0^T\sigma_t^2 \;dt\right] = 2\left(\int_0^{F}\frac{P(K)}{K^2}\;dK + \int_{F}^{\infty}\frac{C(K)}{K^2}\;dK\right)
    \end{split}
\end{equation*}

In [21]:
def SABR(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

def SABRCall(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesCall(S, K, r, sabr_vol, T)


def SABRPut(S, K, r, alpha, beta, rho, nu, T):
    sabr_vol = SABR(S*np.exp(r*T), K, T, alpha, beta, rho, nu)
    return BlackScholesPut(S, K, r, sabr_vol, T)


def sabrcallintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRCall(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price


def sabrputintegrand(K, S, r, T, alpha, beta, rho, nu):
    price = SABRPut(S, K, r, alpha, beta, rho, nu, T) / K**2
    return price

In [24]:
S = 3663.3762493669747
T = 0.1232876712328767
r = 0.002206
K = 3663.3898755765863
alpha = 2.137298210993995
beta = 0.7
rho = -0.6008211054152619
nu = 2.2665005058006495

def SABRPayoff(K, S, r, T, alpha, beta, rho, nu):
    I_put = quad(lambda x: sabrputintegrand(x, S, r, T, alpha, beta, rho, nu), 1e-6, S * np.exp(r*T))
    I_call = quad(lambda x: sabrcallintegrand(x, S, r, T, alpha, beta, rho, nu), S * np.exp(r*T), 5000)
    payoff = 2*np.exp(r*T)*(I_put[0] + I_call[0])
    return payoff
    
SABRPayoff(K, S, r, T, alpha, beta, rho, nu)

0.006322994300468699